In [1]:
from kafka import KafkaConsumer

In [ ]:
consumer = KafkaConsumer(
    'metrics',                       
    bootstrap_servers=['localhost:9092'], 
    auto_offset_reset='earliest',        
    enable_auto_commit=True,             
    group_id='my-consumer-group'
)


Consumer is now listening to 'metrics'...


In [ ]:
import re
from confluent_kafka import Consumer
import mysql.connector


conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'metrics-group',
    'auto.offset.reset': 'earliest'
}

consumer = Consumer(conf)
consumer.subscribe(['metrics'])

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="rootpass",
    database="metrics_logs"
)
cursor = conn.cursor()

print("Consumer is now listening to 'metrics'...")

pattern = r'id:\s*(?P<id>\d+),cpu:\s*(?P<cpu>\d+),mem:\s*(?P<mem>\d+),disk:\s*(?P<disk>\d+)'

try:
    while True:
        msg = consumer.poll(1.0)
        if msg is None:
            continue
        if msg.error():
            print("Error: {}".format(msg.error()))
            continue

        metric_line = msg.value().decode('utf-8')
        print("Received:", metric_line)

        match = re.match(pattern, metric_line)
        if match:
            data = match.groupdict()

            cursor.execute(
                "INSERT INTO metrics (server_id, cpu_usage, mem_usage, disk_usage) VALUES (%s, %s, %s, %s)",
                (
                    data["id"],
                    data["cpu"],
                    data["mem"],
                    data["disk"]
                )
            )
            conn.commit()
        else:
            print("Failed to parse metric line:", metric_line)
except KeyboardInterrupt:
    print("Exiting...")
finally:
    consumer.close()
    cursor.close()
    conn.close()

Consumer is now listening to 'metrics'...
Received: id: 8,cpu: 95,mem: 70,disk: 59
Received: id: 3,cpu: 5,mem: 10,disk: 35
Received: id: 2,cpu: 83,mem: 78,disk: 40
Received: id: 6,cpu: 62,mem: 44,disk: 26
Received: id: 0,cpu: 56,mem: 96,disk: 40
Received: id: 1,cpu: 75,mem: 91,disk: 29
Received: id: 7,cpu: 46,mem: 10,disk: 63
Received: id: 4,cpu: 87,mem: 94,disk: 92
Received: id: 5,cpu: 63,mem: 32,disk: 64
Received: id: 6,cpu: 63,mem: 40,disk: 24
Received: id: 5,cpu: 38,mem: 74,disk: 46
Received: id: 2,cpu: 13,mem: 29,disk: 3
Received: id: 4,cpu: 29,mem: 33,disk: 20
Received: id: 8,cpu: 81,mem: 87,disk: 18
Received: id: 7,cpu: 64,mem: 90,disk: 31
Received: id: 9,cpu: 73,mem: 28,disk: 53
Received: id: 3,cpu: 78,mem: 75,disk: 42
Received: id: 0,cpu: 19,mem: 92,disk: 31
Received: id: 1,cpu: 79,mem: 51,disk: 66
Received: id: 2,cpu: 40,mem: 57,disk: 17
Received: id: 5,cpu: 23,mem: 37,disk: 19
Received: id: 6,cpu: 9,mem: 39,disk: 12
Received: id: 4,cpu: 94,mem: 39,disk: 32
Received: id: 8,cp